In [1]:
import numpy as np
import cv2 as cv
import os

from tqdm import tqdm

In [2]:
def getFiles(PATH):
    _files = []
    for root, dirs, files in os.walk(PATH):
        for fichero in files:
            name, extension = os.path.splitext(fichero)
            if extension == ".jpg":
                _files.append(os.path.join(PATH,fichero))
    return _files

In [3]:
centers = np.load('centers.npy')
n_centers = centers.shape[0]

In [4]:
ficheros= getFiles('.//..//..//MSCOCO//train2014')

In [5]:
N_SAVE = 1000
sift = cv.xfeatures2d.SIFT_create()
VLAD_DESCRIPTORS = []

i=1
for fichero in tqdm(ficheros):
    img = cv.imread(fichero)
    img = cv.resize(img,(224,224))
    gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray,None)
    VLAD = np.zeros((n_centers,128))
    
    if des is not None:
    
        for d in des:
            local_dist = []
            for c in centers:
                local_dist.append( np.sqrt(np.sum(np.square(d-c))) )
            idx = np.where(local_dist == np.min(local_dist))[0][0]
            VLAD[idx] = VLAD[idx] + d-c[idx]
    
    VLAD_DESCRIPTORS.append(VLAD.reshape(1,-1)/np.sqrt(np.sum(np.square(VLAD.reshape(1,-1)))))
    
    if i%N_SAVE == 0:
        VLAD_DESCRIPTORS = np.asarray(VLAD_DESCRIPTORS)
        np.save('vlad_descriptors//vlad_{}'.format(int(i/N_SAVE)), VLAD_DESCRIPTORS)
        VLAD_DESCRIPTORS = []
    
    i = i+1
    

100%|██████████| 82783/82783 [7:18:41<00:00,  3.15it/s]  


In [6]:
np.save('vlad_descriptors//vlad_end',VLAD_DESCRIPTORS)